# Exercise 7

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [17]:
from sklearn.datasets import make_moons

moons = make_moons(n_samples=10000, noise=0.4)

In [18]:
X = moons[0]
y = moons[1]

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state=42)
params = {
    'min_samples_split': [2, 3, 4],
    'max_leaf_nodes': list(range(2, 100))
}

gs = GridSearchCV(tree, params, cv=3, verbose=1)
gs.fit(X_train, y_train)

Fitting 3 folds for each of 294 candidates, totalling 882 fits


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_split': [2, 3, 4]},
             verbose=1)

In [21]:
gs.best_estimator_

DecisionTreeClassifier(max_leaf_nodes=25, random_state=42)

In [22]:
from sklearn.metrics import accuracy_score

y_pred = gs.best_estimator_.predict(X_test)
accuracy_score(y_test, y_pred)

0.8655

# Exercise 8

In [24]:
from sklearn.model_selection import ShuffleSplit

split = ShuffleSplit(n_splits=1000, test_size=len(X_train)-100, random_state=42)

mini_sets = []


for train_index, test_index in split.split(X):
    X_train_mini = X[train_index]
    y_train_mini = y[train_index]
    mini_sets.append((X_train_mini, y_train_mini))

In [25]:
forest = []

for X_train_mini, y_train_mini in mini_sets:
    tree = DecisionTreeClassifier(max_leaf_nodes=25, random_state=42)
    tree.fit(X_train_mini, y_train_mini)
    forest.append(tree)


In [33]:
scores = []

for tree in forest:
    y_pred = tree.predict(X_test)
    scores.append(accuracy_score(y_test, y_pred))

scores = np.array(scores)
np.mean(scores)

0.8663379999999999

In [37]:
from scipy.stats import mode

Y_pred = np.empty([1000, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

Y_pred

array([[1, 1, 0, ..., 0, 0, 1],
       [1, 1, 0, ..., 0, 0, 1],
       [1, 1, 0, ..., 0, 0, 1],
       ...,
       [1, 1, 0, ..., 0, 0, 1],
       [1, 1, 0, ..., 0, 0, 1],
       [1, 1, 0, ..., 0, 0, 1]], dtype=uint8)

In [44]:
most_frequent, n_values = mode(Y_pred, axis=0)

/var/folders/q7/0s4jjvb94hb_2_90j2545plw0000gn/T/ipykernel_31132/2633225482.py:1: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  most_frequent, n_values = mode(Y_pred, axis=0)


(1, 2000)

In [47]:
most_frequent.reshape([-1, 1]).shape

(2000, 1)

In [42]:
accuracy_score(y_test, most_frequent.reshape([-1]))

0.88